### 参考文献
pygribで気象庁のGRIB2ファイルを読む：https://qiita.com/kurukuruz/items/6fc0be9efa34a2fd6741

In [27]:
# coding: utf-8
import os
import subprocess
import pygrib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import wxparams as wx

In [28]:
df_input = pd.read_excel('input.xlsx', dtype=str, index_col=None)
print(df_input)

    latitude   longitude  year month day    time
0  43.626260  144.323616  2023    11  08  000000
1  43.626260  144.323616  2023    11  09  120000
2  43.626260  144.323616  2023    11  10  060000


In [29]:
latitude_ls = list(df_input['latitude'])
latitude_ls = [float(i) for i in latitude_ls]
longitude_ls = list(df_input['longitude'])
longitude_ls = [float(i) for i in longitude_ls]
year_ls = list(df_input['year'])
month_ls = list(df_input['month'])
day_ls = list(df_input['day'])
time_ls = list(df_input['time'])
size = df_input.shape[0]

In [30]:
cwd = os.getcwd()

# GSM
# url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2020/08/01/Z__C_RJTD_20200801060000_GSM_GPV_Rjp_Lsurf_FD0000-0312_grib2.bin'

# MSM
# url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2023/08/02/Z__C_RJTD_20230802000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin'
url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2023/08/02/Z__C_RJTD_20230802000000_MSM_GPV_Rjp_Lsurf_FH16-33_grib2.bin'
# url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2023/08/02/Z__C_RJTD_20230802000000_MSM_GPV_Rjp_Lsurf_FH34-39_grib2.bin'
# url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2023/08/02/Z__C_RJTD_20230802000000_MSM_GPV_Rjp_Lsurf_FH40-51_grib2.bin'
# url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2023/08/02/Z__C_RJTD_20230802000000_MSM_GPV_Rjp_Lsurf_FH52-78_grib2.bin'

# ダウンロード
# subprocess.run(['curl', '-O', url_surf], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=cwd)

In [31]:
## GRIB2ファイルを読み込み，変数に格納して出力する関数
def readgrib2(url_surf):
    file_surf = os.path.join(cwd, os.path.basename(url_surf))
    grbs = pygrib.open(file_surf)
    return(file_surf, grbs)

# file_surf, grbs = readgrib2(url_surf)

In [32]:
for k in range(size):
    latitude = latitude_ls[k]
    longitude = longitude_ls[k]
    time_diff = timedelta(hours=9) # UTCとJSTの差分
    df = pd.DataFrame()

    datatype = "MSM"
    year = year_ls[k]
    month = month_ls[k]
    date = day_ls[k]
    time = time_ls[k]
    initialtime = f"{year}{month}{date}{time}"
    cwd = os.getcwd()

    url_list = [f'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/{year}/{month}/{date}/Z__C_RJTD_{initialtime}_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin'\
    ,f'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/{year}/{month}/{date}/Z__C_RJTD_{initialtime}_MSM_GPV_Rjp_Lsurf_FH16-33_grib2.bin'\
    ,f'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/{year}/{month}/{date}/Z__C_RJTD_{initialtime}_MSM_GPV_Rjp_Lsurf_FH34-39_grib2.bin'\
    ,f'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/{year}/{month}/{date}/Z__C_RJTD_{initialtime}_MSM_GPV_Rjp_Lsurf_FH40-51_grib2.bin'\
    ,f'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/{year}/{month}/{date}/Z__C_RJTD_{initialtime}_MSM_GPV_Rjp_Lsurf_FH52-78_grib2.bin']

    if time != "000000" and time != "120000":
        url_list = url_list[0:3]

    for url_surf in url_list:
        subprocess.run(['curl', '-O', url_surf], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=cwd) # データダウンロード
        file_surf, grbs = readgrib2(url_surf)
        # データを取り出す
        prmsl = grbs.select(parameterName='Pressure reduced to MSL')
        sp    = grbs.select(parameterName='Pressure')
        uwind = grbs.select(parameterName='u-component of wind')
        vwind = grbs.select(parameterName='v-component of wind')
        temp  = grbs.select(parameterName='Temperature')
        rh    = grbs.select(parameterName='Relative humidity')
        lcc   = grbs.select(parameterName='Low cloud cover')
        mcc   = grbs.select(parameterName='Medium cloud cover')
        hcc   = grbs.select(parameterName='High cloud cover')
        tcc   = grbs.select(parameterName='Total cloud cover')
        tp    = grbs.select(parameterName='Total precipitation')
        dswrf = grbs.select(parameterName='Downward short-wave radiation flux')

        df1d = pd.DataFrame({
            "validDate": [msg.validDate + time_diff for msg in temp],
            "temperature": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] - 273.15 for msg in temp
            ],
            "rh": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in rh
            ],
            "prmsl": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in prmsl
            ],
            "sp": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in sp
            ],
            "uwind": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in uwind
            ],
            "vwind": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in vwind
            ],
            "lcc": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in mcc
            ],
            "mcc": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in rh
            ],
            "hcc": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in hcc
            ],
            "tcc": [
                msg.data(
                    lat1=latitude-0.025,
                    lat2=latitude+0.025,
                    lon1=longitude-0.03125,
                    lon2=longitude+0.03125,
                )[0][0][0] for msg in tcc
            ],
            # "dswrf": [
            #     msg.data(
            #         lat1=latitude-0.025,
            #         lat2=latitude+0.025,
            #         lon1=longitude-0.03125,
            #         lon2=longitude+0.03125,
            #     )[0][0][0] for msg in dswrf
            # ]
        })
        print(df1d)
        df = pd.concat([df, df1d])

    #　風速をU, V成分から風向風速に変換する
    U = df["uwind"]
    V = df["vwind"]
    Wspd, Wdir = wx.UV_to_SpdDir(U, V) # UV_to_SpdDir(U, V)
    df["windspeed"] = Wspd
    df["winddirection"] = Wdir

    print(df)
    df.to_csv(f'{latitude}_{longitude}_{initialtime}.csv')

             validDate  temperature         rh          prmsl            sp  \
0  2023-11-08 09:00:00     6.684839  79.915813  101808.966064  98652.056885   
1  2023-11-08 10:00:00     6.489832  78.164179  101930.017090  98772.692871   
2  2023-11-08 11:00:00     6.571863  74.937185  102036.285400  98866.894531   
3  2023-11-08 12:00:00     6.500177  72.106979  102094.488525  98922.265625   
4  2023-11-08 13:00:00     6.635492  71.661198  102115.454102  98944.018555   
5  2023-11-08 14:00:00     7.023859  69.039148  102158.197021  98992.071533   
6  2023-11-08 15:00:00     6.845392  68.285170  102257.330322  99084.619141   
7  2023-11-08 16:00:00     6.377496  70.836658  102360.321045  99188.183594   
8  2023-11-08 17:00:00     5.957422  75.650900  102465.667725  99276.745605   
9  2023-11-08 18:00:00     5.803979  78.216805  102555.072021  99362.615967   
10 2023-11-08 19:00:00     5.631006  79.402822  102611.450195  99421.057129   
11 2023-11-08 20:00:00     5.512964  79.511650  1026

In [ ]:
pd.concat([df, df])

In [ ]:
grbs

In [ ]:
grbs[1]

In [ ]:
grbs[1].data(
        lat1=latitude-0.025,
        lat2=latitude+0.025,
        lon1=longitude-0.03125,
        lon2=longitude+0.03125,)[0][0][0]

In [ ]:
grbs[1].data(
        lat1=latitude-0.025,
        lat2=latitude+0.025,
        lon1=longitude-0.03125,
        lon2=longitude+0.03125,)

In [ ]:
grbs[1].validDate

In [ ]:
# GRIB2ファイルの中身を表示する
for grb in grbs:
    print(grb)

In [ ]:
for grb in grbs:
    print(grb.name)

In [ ]:
value = []
# for i in range(0,15+1):
# for i in range(16,33+1):
# for i in range(34,39+1):
# for i in range(40,51+1):
for i in range(52,78+1):
    print(i)
    # 北緯34度から36度、東経135度から140度のデータを取り出す
    # prmsl_fc0 = grbs.select(parameterName='Temperature', forecastTime=i)[0]
    prmsl_fc0 = grbs.select(parameterName='Relative humidity', forecastTime=i)[0]
    values, lats, lons = prmsl_fc0.data(lat1=43.5, lat2=43.7, lon1=144.0, lon2=145.0)
    # print(values)
    # print(lats)
    # print(lons)
    value.append(values[1,8])
value

In [ ]:
pd.DataFrame(value).to_csv('Relative-humidity_5.csv')

In [ ]:
# 指定した観測データを取り出す
prmsl = grbs.select(parameterName='Pressure reduced to MSL')
sp    = grbs.select(parameterName='Pressure')
uwind = grbs.select(parameterName='u-component of wind')
vwind = grbs.select(parameterName='v-component of wind')
temp  = grbs.select(parameterName='Temperature')
rh    = grbs.select(parameterName='Relative humidity')
lcc   = grbs.select(parameterName='Low cloud cover')
mcc   = grbs.select(parameterName='Medium cloud cover')
hcc   = grbs.select(parameterName='High cloud cover')
tcc   = grbs.select(parameterName='Total cloud cover')
tp    = grbs.select(parameterName='Total precipitation')
dswrf = grbs.select(parameterName='Downward short-wave radiation flux')

In [ ]:
temp

In [ ]:
# 初期時刻から12時間後のデータを取り出す
fc12 = grbs.select(forecastTime=12)

In [ ]:
# prmsl_fc0 = grbs.select(parameterName='Temperature', forecastTime=15)[0]
# values, lats, lons = prmsl_fc0.data(lat1=43.5, lat2=43.7, lon1=144.0, lon2=145.0)
# print(values)
# print(lats)
# print(lons)
# value.append(values[1,8])

In [ ]:
for i in [values,lats,lons]:
    print(i.shape)

In [ ]:
pd.DataFrame(values).to_csv('values.csv')
pd.DataFrame(lats).to_csv('lat.csv')
pd.DataFrame(lons).to_csv('lon.csv')

In [ ]:
# GRIB2ファイルを閉じる
grbs.close()

In [ ]:
def inquire_grib_data(path):
    import pygrib                  # gribファイルの中身を見たい場合はinquire_grib_data(path)を実行                
    grbs = pygrib.open(path)
    for grb in grbs:
        print(grb)
    return

def read_grib_data(path,name=None,level=None):
    import numpy as np
    import pygrib                   # pygribは!pip3 install pygrib --userでインストール
    grbs = pygrib.open(path)

    if name != None:                # anl_surf125に対しては変数名を与える
        alines = grbs.select(name=name)
    elif level != None:             # anl_p125に対しては気圧面を与えるとその水平面データ
        alines = grbs.select(level=level)
    else:                           #                  気圧面を与えないと全３次元データ
        alines = grbs.select()

    lat, lon = alines[0].latlons()  # lonは経度、latは緯度データ: (ny,nx)の２次元格子です
    ny, nx = lat.shape
    nline = len(alines)
    gdata = np.empty( (nline,ny,nx), dtype = "f4" )
    levels = np.empty( (nline), dtype = "f4" )
    for iline, aline in enumerate(alines):
        gdata[iline,:,:] = aline.values[::-1,:]
        levels[iline] = aline["level"]

    return lon, lat[::-1], level, gdata 
inquire_grib_data("anl_surf125.2020010100") # これを実行すると、この下のnameに何を与えるかがわかります
lon, lat, _, SLP = read_grib_data("anl_surf125.2020010100",name="Mean sea level pressure")
T850 = read_grib_data("anl_p125_tmp.2020010100",level=850)[3]

In [ ]:
# grbs = pygrib.open("Z__C_RJTD_20171205000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin")

# 項目の重複をなくすため、予測時間=1で絞込
msgs_at1 = grbs.select(forecastTime=1)

tmpList = []
for msg in msgs_at1:
    tmpList.append([msg.parameterCategory, msg.parameterNumber, msg.shortName, msg.name])

print(pd.DataFrame(tmpList, columns=['category', 'number', 'shortName', 'name']))